In [1]:
import os
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.saving import load_model
from skimage.morphology import disk

In [2]:
train_ims = sorted(glob(os.path.join('.\Train\Train','*.jpg')))
print(len(train_ims))

18998


In [3]:
print(train_ims[:5])

['.\\Train\\Train\\ISIC_0000000.jpg', '.\\Train\\Train\\ISIC_0000001.jpg', '.\\Train\\Train\\ISIC_0000002.jpg', '.\\Train\\Train\\ISIC_0000003.jpg', '.\\Train\\Train\\ISIC_0000004.jpg']


In [4]:
seg_ims = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_ims))

1945


In [5]:
for i in range(len(seg_ims)):
    seg_ims[i] = seg_ims[i].replace('_seg.png','.jpg')
print(seg_ims[:5])

['.\\Train\\Train\\ISIC_0000000.jpg', '.\\Train\\Train\\ISIC_0000001.jpg', '.\\Train\\Train\\ISIC_0000003.jpg', '.\\Train\\Train\\ISIC_0000004.jpg', '.\\Train\\Train\\ISIC_0000007.jpg']


In [6]:
undone_train = np.array([x for x in train_ims if x not in seg_ims])
print(undone_train[:5])

['.\\Train\\Train\\ISIC_0000002.jpg' '.\\Train\\Train\\ISIC_0000010.jpg'
 '.\\Train\\Train\\ISIC_0000114.jpg' '.\\Train\\Train\\ISIC_0000118.jpg'
 '.\\Train\\Train\\ISIC_0000174.jpg']


In [7]:
test_ims = sorted(glob(os.path.join('.\Test\Test','*.jpg')))
print(len(test_ims))
print(test_ims[:5])

segtest_ims = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(segtest_ims))

for i in range(len(segtest_ims)):
    segtest_ims[i] = segtest_ims[i].replace('_seg.png','.jpg')
print(segtest_ims[:5])

undone_test = np.array([x for x in test_ims if x not in segtest_ims])
print(undone_test[:5])

6333
['.\\Test\\Test\\ISIC_0000006.jpg', '.\\Test\\Test\\ISIC_0000011.jpg', '.\\Test\\Test\\ISIC_0000014.jpg', '.\\Test\\Test\\ISIC_0000018.jpg', '.\\Test\\Test\\ISIC_0000022.jpg']
648
['.\\Test\\Test\\ISIC_0000006.jpg', '.\\Test\\Test\\ISIC_0000011.jpg', '.\\Test\\Test\\ISIC_0000014.jpg', '.\\Test\\Test\\ISIC_0000018.jpg', '.\\Test\\Test\\ISIC_0000022.jpg']
['.\\Test\\Test\\ISIC_0000130.jpg' '.\\Test\\Test\\ISIC_0000195.jpg'
 '.\\Test\\Test\\ISIC_0000551.jpg' '.\\Test\\Test\\ISIC_0003051.jpg'
 '.\\Test\\Test\\ISIC_0004166.jpg']


In [14]:
filenametrain = []
for i in range(len(undone_train)):
    name = undone_train[i]
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    filenametrain.append(name)
print(filenametrain[:5])

['./Train/Modified/ISIC_0000002.jpg', './Train/Modified/ISIC_0000010.jpg', './Train/Modified/ISIC_0000114.jpg', './Train/Modified/ISIC_0000118.jpg', './Train/Modified/ISIC_0000174.jpg']


In [9]:
filenametest = []
for i in range(len(undone_test)):
    name = undone_test[i]
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    filenametest.append(name)
print(filenametest[:5])

['./Test/Modified/ISIC_0000130.jpg', './Test/Modified/ISIC_0000195.jpg', './Test/Modified/ISIC_0000551.jpg', './Test/Modified/ISIC_0003051.jpg', './Test/Modified/ISIC_0004166.jpg']


In [19]:
kernel = cv.getStructuringElement(1,(9,9))
clahe = cv.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))

for i in range(len(undone_train)):
    if i%100 ==0:
        print(i)
    im = cv.resize(cv.imread(undone_train[i],cv.IMREAD_GRAYSCALE),(512,384))
    blackhat = cv.morphologyEx(im, cv.MORPH_BLACKHAT, kernel)

    bhg= cv.GaussianBlur(blackhat,(3,3),cv.BORDER_DEFAULT)
    _,mask = cv.threshold(bhg,1,255,cv.THRESH_BINARY)

    dst = cv.inpaint(im,mask,6,cv.INPAINT_TELEA)

    mask = cv.dilate((dst<15).astype(np.uint8),disk(30))
    post = cv.inpaint(dst,mask,6,cv.INPAINT_TELEA)

    fim = clahe.apply(post)
    cv.imwrite(filenametrain[i],fim)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [18]:
for i in range(len(undone_test)):
    if i%100 ==0:
        print(i)
    im = cv.resize(cv.imread(undone_test[i],cv.IMREAD_GRAYSCALE),(512,384))
    blackhat = cv.morphologyEx(im, cv.MORPH_BLACKHAT, kernel)

    bhg= cv.GaussianBlur(blackhat,(3,3),cv.BORDER_DEFAULT)
    _,mask = cv.threshold(bhg,1,255,cv.THRESH_BINARY)

    dst = cv.inpaint(im,mask,6,cv.INPAINT_TELEA)

    mask = cv.dilate((dst<15).astype(np.uint8),disk(30))
    post = cv.inpaint(dst,mask,6,cv.INPAINT_TELEA)

    fim = clahe.apply(post)
    cv.imwrite(filenametest[i],fim)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
